In [1]:
# -*- inception deneme -*-

import os
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model


base_dir = 'D:\pokemon small dataset\inceptiondeneme'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_alakazam = os.path.join(train_dir, 'Alakazam')
train_aerodactyl = os.path.join(train_dir, 'Aerodactyl')
train_arcanine = os.path.join(train_dir, 'Arcanine')
train_goldeen = os.path.join(train_dir, 'Goldeen')

validation_alakazam = os.path.join(validation_dir, 'Alakazam')
validation_aerodactyl = os.path.join(validation_dir, 'Aerodactyl')
validation_arcanine = os.path.join(validation_dir, 'Arcanine')
validation_goldeen = os.path.join(validation_dir, 'Goldeen')


# Set up matplotlib fig, and size it to fit 4x4 pics
import matplotlib.image as mpimg
nrows = 4
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index = 100
train_alakazam_fnames = os.listdir( train_alakazam )
train_aerodactyl_fnames = os.listdir( train_aerodactyl )
train_arcanine_fnames = os.listdir( train_arcanine )
train_goldeen_fnames = os.listdir( train_goldeen )


next_alakazam_pix = [os.path.join(train_alakazam, fname) 
                for fname in train_alakazam_fnames[ pic_index-8:pic_index] 
               ]

next_aerodactyl_pix = [os.path.join(train_aerodactyl, fname) 
                for fname in train_aerodactyl_fnames[ pic_index-8:pic_index] 
               ]

next_arcanine_pix = [os.path.join(train_arcanine, fname) 
                for fname in train_arcanine_fnames[ pic_index-8:pic_index] 
               ]

next_goldeen_pix = [os.path.join(train_goldeen, fname) 
                for fname in train_goldeen_fnames[ pic_index-8:pic_index] 
               ]



for i, img_path in enumerate(next_alakazam_pix+next_aerodactyl_pix+next_arcanine_pix+next_goldeen_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


# Import the inception model  
from tensorflow.keras.applications.inception_v3 import InceptionV3

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), # Shape of our images
                                include_top = False, # Leave out the last fully connected layer
                                weights = 'imagenet')

for layer in pre_trained_model.layers:
  layer.trainable = False





from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(pre_trained_model.output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,batch_size = 20,class_mode = 'categorical', target_size = (150, 150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'categorical', 
                                                          target_size = (150, 150))

def myCallback():
    return 0

callbacks = myCallback()
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 1,
            epochs = 5,
            validation_steps = 5,
            verbose = 2)

<Figure size 1600x1600 with 0 Axes>

Found 6541 images belonging to 150 classes.
Found 289 images belonging to 57 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1 steps, validate for 5 steps
Epoch 1/5
1/1 - 29s - loss: 0.7631 - acc: 0.6973 - val_loss: 1.5793 - val_acc: 0.9825
Epoch 2/5
1/1 - 12s - loss: 0.2241 - acc: 0.9933 - val_loss: 1.3911 - val_acc: 0.9825
Epoch 3/5
1/1 - 13s - loss: 0.1984 - acc: 0.9933 - val_loss: 1.2551 - val_acc: 0.9825
Epoch 4/5
1/1 - 12s - loss: 0.1847 - acc: 0.9933 - val_loss: 1.1375 - val_acc: 0.9825
Epoch 5/5
1/1 - 13s - loss: 0.1683 - acc: 0.9933 - val_loss: 1.0267 - val_acc: 0.9825
